In [1]:
!pip install pygame

     ---------------------------------------- 10.6/10.6 MB 4.2 MB/s eta 0:00:00


In [3]:
import cv2
import os
import numpy as np
from pygame import mixer
import time
from keras.models import load_model

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
!pip install twilio

     ---------------------------------------- 1.7/1.7 MB 123.1 kB/s eta 0:00:00
     ------------------------------------- 323.6/323.6 kB 99.8 kB/s eta 0:00:00
     --------------------------------------- 61.7/61.7 kB 68.6 kB/s eta 0:00:00


In [72]:
import os
from twilio.rest import Client
import datetime

account_sid = 'ACngftlrficicjclifcvvfic1dfg8'
auth_token = "0dc5ufuffugtfifv812462"
client = Client(account_sid, auth_token)

def send_sms():
    time = datetime.datetime.now()
    message = client.messages.create(
                              body='Hi\nDriver was Sleepy at {h:02d}:{m:02d}{mode}\nPlease Call them!!!'.format("%2d",h=time.hour%12,m=h=time.minute,mode =('AM' if time.hour<12 else 'PM')),
                              from_='+1416434593',
                              to='+91787645487'
                          )
def call():
    time = datetime.datetime.now()
    call = client.calls.create(
                            twiml='<Response><Say>Hi, Driver is sleepy. Please take action.</Say></Response>',
                            from_='+1416434593',
                            to='+91787645487'
                        )

In [51]:
mixer.init()
sound = mixer.Sound(r"G:\beep-warning-6387.mp3")

model = load_model(r"G:\model.h5")

video = cv2.VideoCapture(0)

frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

result = cv2.VideoWriter(r'G:\filename.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         5, size)
face = cv2.CascadeClassifier(r"G:\haarcascade_frontalface_alt.xml")
leye = cv2.CascadeClassifier(r"G:\haarcascade_lefteye_2splits.xml")
reye = cv2.CascadeClassifier(r"G:\haarcascade_righteye_2splits.xml")

score=0
rpred=[1]
lpred=[1]
call_threshold = 0
while(True):
    
    if(call_threshold>5):
        call_threshold = 0
        send_sms()
        call()
    
    ret, frame = video.read()
    height,width = frame.shape[:2] 

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (0,255,0) , 2 )
        
        left_eye = leye.detectMultiScale(frame[y:y+h,x:x+w])
        right_eye =  reye.detectMultiScale(frame[y:y+h,x:x+w])
        
        for (x1,y1,w1,h1) in right_eye:
            x1+=x
            y1+=y
            r_eye=frame[y1:y1+h1,x1:x1+w1]
            count=count+1
            r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
            r_eye = cv2.resize(r_eye,(64,64))
            r_eye= r_eye/255
            r_eye=  r_eye.reshape(64,64,-1)
            r_eye = np.expand_dims(r_eye,axis=0)
            predict_x=model.predict(r_eye) 
            rpred=np.argmax(predict_x,axis=1)
            break

        for (x2,y2,w2,h2) in left_eye:
            x2+=x
            y2+=y
            l_eye=frame[y2:y2+h2,x2:x2+w2]
            count=count+1
            l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
            l_eye = cv2.resize(l_eye,(64,64))
            l_eye= l_eye/255
            l_eye=l_eye.reshape(64,64,-1)
            l_eye = np.expand_dims(l_eye,axis=0)
            predict_y=model.predict(l_eye) 
            lpred=np.argmax(predict_y,axis=1)
           ## lpred = (model.predict(l_eye) > 0.5).astype("int32") 

            break
            
    font = cv2.FONT_HERSHEY_COMPLEX_SMALL
    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    else:
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
    if(score<0):
        score = 0   
    if(score>10):
        score = 10
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
    if(score>5):
        try:
            sound.play()
            call_threshold+=1
        except:
            pass
        for (x,y,w,h) in faces:
            if(score%2):
                cv2.rectangle(frame, (x,y) , (x+w,y+h) , (0,0,255) , 6 )
            else:
                cv2.rectangle(frame, (x,y) , (x+w,y+h) , (0,0,255) , 2 )
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
          
    cv2.imshow('frame',frame)
    result.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
result.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 27ms/step
